In [8]:
# Dự đoán Bi-GRU + CNN
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, GRU, Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ===== Hàm xây dựng mô hình Bi-GRU + CNN =====
def build_bigru_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(GRU(32, return_sequences=True))(inputs)
    x = Bidirectional(GRU(32, return_sequences=True))(x)

    x = Conv1D(filters=32, kernel_size=5, activation='selu', padding='same')(x)
    x = Conv1D(filters=16, kernel_size=5, activation='selu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(1)(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='mse')
    return model

# ===== Hàm đọc dữ liệu Bitcoin từ file CSV =====
def load_bitcoin_data_csv(path):
    df = pd.read_csv(path)
    columns_needed = ['Open', 'High', 'Low', 'Close', 'Volume', 'Taker Buy Base Volume', 'Taker Buy Quote Volume']
    for col in columns_needed:
        if col not in df.columns:
            raise ValueError(f"Thiếu cột '{col}' trong dữ liệu.")
    df = df[columns_needed].dropna()
    return df

# ===== Hàm chuẩn bị dữ liệu theo chuỗi thời gian =====
def prepare_data(df, time_steps=60):
    scaler_X = MinMaxScaler()
    data_X = scaler_X.fit_transform(df)

    scaler_y = MinMaxScaler()
    data_y = scaler_y.fit_transform(df[['Close']])

    X, y = [], []
    for i in range(time_steps, len(df)):
        X.append(data_X[i-time_steps:i])
        y.append(data_y[i])
    return np.array(X), np.array(y), scaler_X, scaler_y


# ===== Đọc và chuẩn bị dữ liệu =====
path = 'data/BTCUSDT-1d-BINANCE.csv'
data = load_bitcoin_data_csv(path)
X, y, scaler_X, scaler_y = prepare_data(data, time_steps=60)

# ===== Chia dữ liệu train/test =====
split = int(0.75 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ===== Xây dựng và huấn luyện mô hình =====
model = build_bigru_cnn_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# ===== Dự đoán =====
predictions = model.predict(X_test)

# ===== Giải tỉ lệ để đánh giá =====
predictions = scaler_y.inverse_transform(predictions)
y_test_true = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# ===== Đánh giá hiệu suất mô hình =====
mae = mean_absolute_error(y_test_true, predictions)
mse = mean_squared_error(y_test_true, predictions)
r2 = r2_score(y_test_true, predictions)

print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"R² (R-squared score): {r2:.4f}")


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.2445 - val_loss: 0.0087
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0243 - val_loss: 0.0101
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0078 - val_loss: 0.0072
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0076 - val_loss: 0.0047
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0068 - val_loss: 0.0054
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0070 - val_loss: 0.0058
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0088 - val_loss: 0.0041
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0072 - val_loss: 0.0048
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0046 - val_loss: 0.0054
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0064 - val_loss: 0.0035
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0048 - val_loss: 0.0071
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0

In [9]:
#GRU
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ===== Hàm xây dựng mô hình GRU đơn thuần =====
def build_gru_model(input_shape):
    inputs = Input(shape=input_shape)
    x = GRU(64, return_sequences=True)(inputs)
    x = GRU(32)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

# ===== Hàm đọc dữ liệu Bitcoin từ file CSV =====
def load_bitcoin_data_csv(path):
    df = pd.read_csv(path)
    columns_needed = ['Open', 'High', 'Low', 'Close', 'Volume', 'Taker Buy Base Volume', 'Taker Buy Quote Volume']

    for col in columns_needed:
        if col not in df.columns:
            raise ValueError(f"Thiếu cột '{col}' trong dữ liệu.")
    df = df[columns_needed].dropna()
    return df

# ===== Hàm chuẩn bị dữ liệu =====
def prepare_data(df, time_steps=60):
    scaler_X = MinMaxScaler()
    data_X = scaler_X.fit_transform(df)

    scaler_y = MinMaxScaler()
    data_y = scaler_y.fit_transform(df[['Close']])

    X, y = [], []
    for i in range(time_steps, len(df)):
        X.append(data_X[i-time_steps:i])
        y.append(data_y[i])
    return np.array(X), np.array(y), scaler_X, scaler_y

# ===== Load & chuẩn bị dữ liệu =====
path = 'data/BTCUSDT-1d-BINANCE.csv'
data = load_bitcoin_data_csv(path)
X, y, scaler_X, scaler_y = prepare_data(data, time_steps=60)

# ===== Chia train/test =====
split = int(0.75 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ===== Xây dựng & huấn luyện GRU đơn thuần =====
model = build_gru_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# ===== Dự đoán & đánh giá =====
predictions = model.predict(X_test)
predictions_inv = scaler_y.inverse_transform(predictions)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

print(f"MAE: {mean_absolute_error(y_test_inv, predictions_inv):.4f}")
print(f"MSE: {mean_squared_error(y_test_inv, predictions_inv):.4f}")
print(f"R2 : {r2_score(y_test_inv, predictions_inv):.4f}")

Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0907 - val_loss: 0.0102
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0128 - val_loss: 0.0130
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0111 - val_loss: 0.0058
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0104 - val_loss: 0.0027
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0090 - val_loss: 0.0051
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0089 - val_loss: 0.0019
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0085 - val_loss: 0.0108
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0080 - val_loss: 0.0034
Epoch 9/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0092 - val_loss: 0.0032
Epoch 10/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0090 - val_loss: 0.0035
Epoch 11/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0073 - val_loss: 0.0123
Epoch 12/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0

In [10]:
#GRU + CNN
# Dự đoán
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, GRU, Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ===== Hàm xây dựng mô hình GRU + CNN =====
def build_gru_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = GRU(64, return_sequences=True)(inputs)
    x = GRU(64, return_sequences=True)(x)

    x = Conv1D(filters=32, kernel_size=3, activation='selu', padding='same')(x)
    x = Conv1D(filters=16, kernel_size=3, activation='selu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(1)(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='mse')
    return model


# ===== Hàm đọc dữ liệu Bitcoin từ file CSV =====
def load_bitcoin_data_csv(path):
    df = pd.read_csv(path)
    columns_needed = ['Open', 'High', 'Low', 'Close', 'Volume', 'Taker Buy Base Volume', 'Taker Buy Quote Volume']
    for col in columns_needed:
        if col not in df.columns:
            raise ValueError(f"Thiếu cột '{col}' trong dữ liệu.")
    df = df[columns_needed].dropna()
    return df

# ===== Hàm chuẩn bị dữ liệu theo chuỗi thời gian =====
def prepare_data(df, time_steps=60):
    scaler_X = MinMaxScaler()
    data_X = scaler_X.fit_transform(df)

    scaler_y = MinMaxScaler()
    data_y = scaler_y.fit_transform(df[['Close']])

    X, y = [], []
    for i in range(time_steps, len(df)):
        X.append(data_X[i-time_steps:i])
        y.append(data_y[i])
    return np.array(X), np.array(y), scaler_X, scaler_y


# ===== Đọc và chuẩn bị dữ liệu =====
path = 'data/BTCUSDT-1d-BINANCE.csv'
data = load_bitcoin_data_csv(path)
X, y, scaler_X, scaler_y = prepare_data(data, time_steps=60)

# ===== Chia dữ liệu train/test =====
split = int(0.75 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ===== Xây dựng và huấn luyện mô hình =====
model = build_gru_cnn_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# ===== Dự đoán =====
predictions = model.predict(X_test)

# ===== Giải tỉ lệ để đánh giá =====
predictions = scaler_y.inverse_transform(predictions)
y_test_true = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# ===== Đánh giá hiệu suất mô hình =====
mae = mean_absolute_error(y_test_true, predictions)
mse = mean_squared_error(y_test_true, predictions)
r2 = r2_score(y_test_true, predictions)

print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"R² (R-squared score): {r2:.4f}")


Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0807 - val_loss: 0.0045
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0107 - val_loss: 0.0050
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0062 - val_loss: 0.0056
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0058 - val_loss: 0.0076
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0060 - val_loss: 0.0038
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0075 - val_loss: 0.0148
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0062 - val_loss: 0.0036
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0046 - val_loss: 0.0045
Epoch 9/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0039 - val_loss: 0.0057
Epoch 10/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0041 - val_loss: 0.0033
Epoch 11/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0042 - val_loss: 0.0037
Epoch 12/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0

In [12]:
# CNN+BiGRU + EarlyStopping + ReduceLR
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, GRU, Conv1D, MaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ===== Hàm xây dựng mô hình CNN + BiGRU =====
def build_cnn_bigru_model(input_shape):
    inputs = Input(shape=input_shape)

    # CNN để học pattern ngắn hạn
    x = Conv1D(filters=64, kernel_size=5, activation='selu', padding='same')(inputs)
    x = Conv1D(filters=64, kernel_size=5, activation='selu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # BiGRU để học quan hệ dài hạn
    x = Bidirectional(GRU(64, return_sequences=False))(x)

    # Dense & Dropout
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

# ===== Hàm đọc dữ liệu Bitcoin từ file CSV =====
def load_bitcoin_data_csv(path):
    df = pd.read_csv(path)
    columns_needed = ['Open', 'High', 'Low', 'Close', 'Volume', 'Taker Buy Base Volume', 'Taker Buy Quote Volume']
    for col in columns_needed:
        if col not in df.columns:
            raise ValueError(f"Thiếu cột '{col}' trong dữ liệu.")
    df = df[columns_needed].dropna()
    return df

# ===== Hàm chuẩn bị dữ liệu theo chuỗi thời gian =====
def prepare_data(df, time_steps=60):
    scaler_X = MinMaxScaler()
    data_X = scaler_X.fit_transform(df)

    scaler_y = MinMaxScaler()
    data_y = scaler_y.fit_transform(df[['Close']])

    X, y = [], []
    for i in range(time_steps, len(df)):
        X.append(data_X[i-time_steps:i])
        y.append(data_y[i])
    return np.array(X), np.array(y), scaler_X, scaler_y

# ===== Đọc và chuẩn bị dữ liệu =====
path = 'data/BTCUSDT-1d-BINANCE.csv'
data = load_bitcoin_data_csv(path)
X, y, scaler_X, scaler_y = prepare_data(data, time_steps=60)

# ===== Chia dữ liệu train/test =====
split = int(0.75 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# ===== Tạo mô hình =====
model = build_cnn_bigru_model((X_train.shape[1], X_train.shape[2]))

# ===== Callbacks: EarlyStopping + ReduceLR =====
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,             # chờ 5 epoch không giảm thì dừng
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,             # giảm LR 1/2
    patience=3,             # chờ 3 epoch không giảm
    min_lr=1e-6
)

# ===== Huấn luyện mô hình với Callbacks =====
history = model.fit(
    X_train, y_train,
    epochs=50,              # cho số lớn, EarlyStopping tự dừng
    batch_size=64,          # batch_size lớn hơn để nhanh hơn
    validation_data=(X_test, y_test),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ===== Dự đoán =====
predictions = model.predict(X_test)

# ===== Giải tỉ lệ =====
predictions = scaler_y.inverse_transform(predictions)
y_test_true = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# ===== Đánh giá =====
mae = mean_absolute_error(y_test_true, predictions)
mse = mean_squared_error(y_test_true, predictions)
r2 = r2_score(y_test_true, predictions)

print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"R² (R-squared score): {r2:.4f}")


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.1117 - val_loss: 0.0174 - learning_rate: 0.0010
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0178 - val_loss: 0.0123 - learning_rate: 0.0010
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0123 - val_loss: 0.0101 - learning_rate: 0.0010
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0081 - val_loss: 0.0085 - learning_rate: 0.0010
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0062 - val_loss: 0.0040 - learning_rate: 0.0010
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0043 - val_loss: 0.0038 - learning_rate: 0.0010
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0042 - val_loss: 0.0033 - learning_rate: 0.0010
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0059 - val_loss: 0.0146 - learning_rate: 0.0010
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0058 - val_loss: 0.0108 - learning_rate: 0.0010
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 